In [1]:
import numpy as np
from math import sin, cos, atan, sqrt, pi, exp
from bokeh.plotting import figure, output_file, output_notebook, show, curdoc
from bokeh.models import ColumnDataSource, Slider, CustomJS
from bokeh.layouts import column, row, widgetbox

#output_notebook()

In [4]:
# RMS value of voltage
u = 230 

# time vector
t = np.linspace(0,0.08, 100)

# frequency & angular frequency
f = 50
omega = 2 * pi * f

# Resitance
R = 5

# Inductance
L = 0.1
XL = 2*pi*f*L

# Phase angle
phi=atan(XL/R)

# closing angle [rad]
alpha = 0

# Phase A
# Current response
ia = [(sqrt(2)*u/(sqrt(R**2+XL**2))*(sin(omega*k+alpha-phi)-sin(alpha-phi)*exp(-R/L*k))) for k in t]

# DC component of the current
iadc = [(sqrt(2)*u/(sqrt(R**2+XL**2))*-sin(alpha-phi)*(exp(-R/L*k))) for k in t]

# AC steady state current
iau = [(sqrt(2)*u/(sqrt(R**2+XL**2))*sin(omega*k+alpha-phi)) for k in t]

# Phase B
# Current response
ib = [(sqrt(2)*u/(sqrt(R**2+XL**2))*(sin(omega*k+alpha-phi+4*pi/3)-sin(alpha-phi+4*pi/3)*exp(-R/L*k))) for k in t]

# DC component of the current
ibdc = [(sqrt(2)*u/(sqrt(R**2+XL**2))*-sin(alpha-phi+4*pi/3)*(exp(-R/L*k))) for k in t]

# AC steady state current
ibu = [(sqrt(2)*u/(sqrt(R**2+XL**2))*sin(omega*k+alpha-phi+4*pi/3)) for k in t]

# Phase C
# Current response
ic = [(sqrt(2)*u/(sqrt(R**2+XL**2))*(sin(omega*k+alpha-phi+2*pi/3)-sin(alpha-phi+2*pi/3)*exp(-R/L*k))) for k in t]

# DC component of the current
icdc = [(sqrt(2)*u/(sqrt(R**2+XL**2))*-sin(alpha-phi+2*pi/3)*(exp(-R/L*k))) for k in t]

# AC steady state current
icu = [(sqrt(2)*u/(sqrt(R**2+XL**2))*sin(omega*k+alpha-phi+2*pi/3)) for k in t]

# plotting
output_file('3phase_cla.html')
source = ColumnDataSource(data={'t': t, 'ia': ia, 'iadc': iadc, 'iau': iau, 'ib': ib, 'ibdc': ibdc, 'ibu': ibu,
                               'ic': ic, 'icdc': icdc, 'icu': icu})

p = figure(plot_width=800, plot_height=400, title='Phase currents')
p.line('t', 'ia', source=source, legend='Phase A', color='firebrick', line_width=3, line_alpha=0.6)
p.line('t', 'ib', source=source, legend='Phase B', color='orange', line_width=3, line_alpha=0.6)
p.line('t', 'ic', source=source, legend='Phase C', line_width=3, line_alpha=0.6)
p.xaxis.axis_label='Time [s]'
p.yaxis.axis_label='Current [A]'

'''def callback1(source=source, window=None):
    data = source.data
    alpha = cb_object.value
    ia, ib, ic, t = data['ia'], data['ib'], data['ic'], data['t']
    for i in range(len(t)):
        ia[i] = (sqrt(2)*window.u/(sqrt(window.R**2+window.XL**2))*(sin(window.omega*t[i]+alpha-window.phi)-sin(alpha-window.phi)*exp(-window.R/window.L*t[i])))
        ib[i] = (sqrt(2)*u/(sqrt(R**2+XL**2))*(sin(omega*t[i]+alpha-phi+4*pi/3)-sin(alpha-phi+4*pi/3)*exp(-R/L*t[i])))
        ic[i] = (sqrt(2)*u/(sqrt(R**2+XL**2))*(sin(omega*t[i]+alpha-phi+2*pi/3)-sin(alpha-phi+2*pi/3)*exp(-R/L*t[i])))
    source.change.emit() '''
    
callback = CustomJS(args=dict(source=source), code="""
        
    //Rewriting values for JavaScript code
    var R = 5;
    var L = 0.1;
    var f = 50;
    var XL = 2*Math.PI*f*L;
    var omega = 2*Math.PI*f;
    var phi = Math.atan(XL/R);
    var u_n = 230
    
    // get data source from Callback args
    var data = source.data;   
    var alpha = cb_obj.value;
    
    
    
    //Indicating which part of the source are certain values
    t = data['t'];
    ia = data['ia'];
    ib = data['ib'];
    ic = data['ic'];
   
    
    for (i=0; i < t.length; i++) {
        ia[i]= Math.sqrt(2)*u_n/(Math.sqrt(Math.pow(R,2)+ Math.pow(XL,2)))*(Math.sin(omega*t[i]+alpha-phi)-Math.sin(alpha-phi)*Math.exp(-R/L*t[i]));
        ib[i]= Math.sqrt(2)*u_n/(Math.sqrt(Math.pow(R,2)+ Math.pow(XL,2)))*(Math.sin(omega*t[i]+alpha-phi+4*Math.PI/3)-Math.sin(alpha-phi+4*Math.PI/3)*Math.exp(-R/L*t[i]));
        ic[i]= Math.sqrt(2)*u_n/(Math.sqrt(Math.pow(R,2)+ Math.pow(XL,2)))*(Math.sin(omega*t[i]+alpha-phi+2*Math.PI/3)-Math.sin(alpha-phi+2*Math.PI/3)*Math.exp(-R/L*t[i]));
    };    
    
    source.change.emit();

    """)
    
slider = Slider(start=0, end=pi, value=0, step=.1, 
                 title="Phase A voltage closing angle [rad]", callback=callback)

layout = row(slider, p)
show(layout)